In [1]:
!pip install torch transformers pandas numpy matplotlib scikit-learn nltk gensim tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/ba/7c/b971f2485155917ecdcebb210e021e36a6b65457394590be01cc61515310/tensorflow-2.13.0-cp310-cp310-win_amd64.whl.metadata
  Using cached tensorflow-2.13.0-cp310-cp310-win_amd64.whl.metadata (2.6 kB)
  Obtaining dependency information for tensorflow-intel==2.13.0 from https://files.pythonhosted.org/packages/40/fa/98115f6fe4d92e1962f549917be2dc8e369853b7e404191996fedaaf4dd6/tensorflow_intel-2.13.0-cp310-cp310-win_amd64.whl.metadata
  Using cached tensorflow_intel-2.13.0-cp310-cp310-win_amd64.whl.metadata (4.1 kB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Obtaining dependency information for h5py>=2.9.0 from https://files.pythonhosted.org/packages/e2/c4/6f8dae1530d57a6122fd5b72c750187484acbe612f630cb2179e4bcb12c1/

In [18]:
# reload packages
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
import pandas as pd
import numpy as np
import os
import sys
from semaxis import CoreUtil
from semaxis import SemAxis
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
%matplotlib inline
import logging

09-19 19:35 matplotlib.pyplot DEBUG    Loaded backend module://matplotlib_inline.backend_inline version unknown.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger(__name__)

In [20]:
# Load the pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

09-19 19:35 urllib3.connectionpool DEBUG    Resetting dropped connection: huggingface.co
09-19 19:35 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/vocab.txt HTTP/1.1" 200 0


In [21]:
# load bert model
from transformers import BertModel

model = BertModel.from_pretrained("../models")

In [22]:
sa = SemAxis(model, axes_str=CoreUtil.load_wordnet_antonyms_axes())

09-19 19:35 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/vocab.txt HTTP/1.1" 200 0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\elias\anaconda3\envs\nlp\lib\site-packages\transformers\tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of 

In [23]:
sa.compute_document_mean_with_tf(
    ["I like to eat apples and oranges."], min_freq=1
)[0]

array([ 0.04913395,  0.0227672 , -0.05149904,  0.02035379, -0.07861039,
        0.08900583, -0.09128648,  0.00758144,  0.03420597, -0.07418875],
      dtype=float32)

In [24]:
data = []

# get list of file path from data\data\en\dev-articles-subtask-2\*
articles = os.listdir(f"../data/data/en/dev-articles-subtask-2")

for article in articles:
    with open(
        f"../data/data/en/dev-articles-subtask-2/{article}", "r", encoding="utf-8"
    ) as f:
        obj = []

        article_id = article.split(".")[0].replace("article", "")

        obj.append(article_id)

        # read line 3 to n
        lines = f.readlines()[2:]
        obj.append("".join(lines))

        data.append(obj)

# create pandas dataframe
df_articles = pd.DataFrame(data, columns=["article_id", "article"])

# article_id to int
df_articles["article_id"] = df_articles["article_id"].astype(int)

In [25]:
COLUMNS = [str(c) for c in sorted(sa.axes.keys()) if len(c) == 2]

with open("big_table_by_average.tsv", "w") as fo_a:
    fo_a.write("{}\t{}\n".format("article_id", "\t".join(COLUMNS)))

    for index, row in df_articles[0:2].iterrows():

        if index % 100 == 0:
            print(index)

        article_id = row["article_id"]
        text = row["article"]
        mean = sa.compute_document_mean_with_tf([text], min_freq=1)[0]
        fo_a.write("{}\t{}\n".format(article_id, "\t".join([str(v) for v in mean])))

0


c:\Users\elias\anaconda3\envs\nlp\lib\site-packages\transformers\tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


# Bootstrapping

Cannot bootstrap as I need a opposite type of data. So e.g. analyze positive and see how the negative looks like. But I have no info about the type of data. So I cannot bootstrap.

> Skip for now

In [30]:
N = 1000
mode = "average"
logger.info(mode)
try:
    os.mkdir(mode)
except:
    pass

09-19 19:39 __main__     INFO     average


In [31]:
def bootstrapping(df, mode, COLUMNS):
    for aspect in df['aspect'].unique():
        for sentiment in ['positive', 'negative']:
            df_r_count = df.query('aspect == @aspect and sentiment == @sentiment').shape[0]
            with open("{}/bootstrap_{}_{}.tsv".format(mode, aspect.replace("/", "-"), sentiment), "w") as fo:
                fo.write("{}\n".format("\t".join(COLUMNS)))
#                 A = df.query('aspect == @aspect').drop(columns=[c for c in df.columns if '(' not in c]).values
                A = df.drop(columns=[c for c in df.columns if '(' not in c]).values

                for i in range(N):
                    if i % 500 == 0:
                        logger.info("{}-{}-{}".format(aspect, sentiment, i))
                    fmean = np.mean(A[np.random.choice(A.shape[0], df_r_count, replace=False), :], axis=0)
                    fo.write("{}\n".format("\t".join([str(v) for v in (fmean)])))

In [32]:
df = pd.read_csv("big_table_by_{}.tsv".format(mode), sep="\t").dropna()
COLUMNS = [c for c in df.columns if '(' in c]
len(COLUMNS)

10

In [33]:
bootstrapping(df, mode, COLUMNS)

KeyError: 'aspect'